In [ ]:
import microsoft_graph_authenticator.graphrest as graphrest
import xml.etree.ElementTree as ET
import zipfile
import tidylib
import os
import bottle
import json
import glob
import html
import re
import time

## Parameters

In [ ]:
# backup folder from REST Backup
restdir="./data/RestBackup"
exportdir="./data/ExportBackup"

## correct content

In [ ]:
# converts the cobocard to regular inline css style
def convertstyle(content):
    content = content.replace('class="small"', 'style="font-size:8pt"')
    content = content.replace('class="big"',   'style="font-size:21pt"')
    content = content.replace('class="r"',     'style="color:red"')
    content = content.replace('class="g"',     'style="color:green"')
    content = content.replace('class="m"',     'style="color:white;background-color:blue"')
    content = content.replace('<br />','<p></p>')
    content = content.replace('&nbsp;','&#160;')
    content = content.replace('border="0" style="vertical-align:-2px;">', '/>') 
    return content

# remove breaks from title
def convertstyletitle(title):
    title = title.replace('<br />', '&#160;')
    return title

# gets the tex source
def gettexsource(cardsetid, cardnr, texnr, src):
    # walk through all files in export directory
    for subdir, dirs, files in os.walk(exportdir):
        for file in files:
            filename, ext = os.path.splitext(file)
            if ".zip" == ext:
                if "CoboCards_" + cardsetid == filename:
                    with zipfile.ZipFile(subdir + "/" + file) as zipf:
                        # open correct xml file
                        with zipf.open("set.xml") as setxml:
                            tree = ET.parse(setxml)
                            root = tree.getroot()
                            # get content of correct card
                            content = root.findall('card')[int(cardnr)-1].get(src)
                            # create regex expression to find formulars
                            p = re.compile(r'##[^#]+##')
                            # find all formulars and remove #s
                            result = p.findall(content)[texnr][2:-2]
                            return str(result)

# correct images in content
def correctimage(text, images_dict, cardsetid, cardnr, src):
    text = '<div>' + text + '</div>' # xml needs some sort of root element
    text = tidylib.tidy_document(text, {"input_xml": True})[0] # tidy up some problems
    root = ET.fromstring(text)
    tex = 0
    for i in root.iter('img'):
        # normal picture
        if re.match(r'\[Pic\d\]', i.attrib["alt"]): 
            i.attrib["src"] = images_dict[i.attrib["alt"]]
        # tex formula
        else:
            i.attrib.pop("alt") # remove alt attribute
            i.attrib.pop("src") # remove src attribute
            i.tag = "span" # change tag from img to span
            i.set("style","font-family:Courier New") # add style
            i.text = gettexsource(cardsetid, cardnr, tex, src) # get tex source
            tex = tex + 1
    result = ET.tostring(root).decode()
    result = result.replace("&amp;", "&") # fix ampersand
    return result                     

## construct content

In [ ]:
# creates an html page
def createpagehtml(title, content, images_dict, cardsetid, cardnr):
    # encode and decode in order to fix problems with encoding
    title = title.encode('ascii', 'xmlcharrefreplace').decode()
    content = content.encode('ascii', 'xmlcharrefreplace').decode()
    
    # convert style
    title = convertstyle(convertstyletitle(title))
    content = convertstyle(content)
    
    # correct images
    title = correctimage(title, images_dict, cardsetid, cardnr, 'title')
    content = correctimage(content, images_dict, cardsetid, cardnr, 'content')
    
    # create data
    data = f"\
<!DOCTYPE html>\r\n\
<html>\r\n\
<head>\r\n\
<title>{title}</title>\r\n\
</head>\r\n\
<body data-absolute-enabled=\"true\" style=\"font-family:Arial;font-size:16pt\">\r\n\
<div style=\"position:absolute;left:50px;top:200px;width:600px\"><p></p>\r\n\
{content}\r\n\
</div>\r\n\
</body>\r\n\
</html>"
    return data

# creates an onenote page
def createpage(title, content, images_dict, cardsetid, card, boundary):
    data = f"\
--{boundary}\r\n\
Content-Disposition:form-data; name=\"Presentation\" \r\n\
Content-Type:text/html\r\n\
{createpagehtml(title, content, images_dict, cardsetid, card)}\r\n\
--{boundary}--"
    return data

## import content

In [ ]:
MSGRAPH = graphrest.GraphSession()

# do the actual importing process               
def importcards():
    notebooksendpoint = MSGRAPH.api_endpoint('me/onenote/notebooks')         
    
    # walk through all files 
    for subdir, dirs, files in os.walk(restdir):
        
        # remove trailing slash if exists
        if subdir.endswith('/'):
            subdir = subdir[:-1]
            
        # loop over all files
        for file in files:
            filename, ext = os.path.splitext(file)
            if ".xml" == ext:
                
                # get basic identifiers
                cardsetid = filename
                setname = os.path.split(subdir)[1]
                foldername = os.path.split(os.path.split(subdir)[0])[1]

                # get existing notebooks
                notebooks = MSGRAPH.get(notebooksendpoint).json()
                notebookid = ""
                for notebook in notebooks["value"]:
                    if foldername == notebook["displayName"]:
                        notebookid = notebook["id"]
                        break

                # create new notebook if not exists
                if "" == notebookid:
                    time.sleep(10)
                    print("create new folder: " + foldername)
                    data = {"displayName": foldername}
                    res = MSGRAPH.post(endpoint=notebooksendpoint, data=json.dumps(data))
                    if 201 != res.status_code:
                        print(res.json())
                        return "failed with status code: " + str(res.status_code)
                    resjson = res.json()
                    notebookid = resjson["id"] 
                
                with open(subdir + "/" + file) as xmlfile:
                    tree = ET.parse(xmlfile)
                    root = tree.getroot()
                    
                    # get section list
                    sectionsendpoint = MSGRAPH.api_endpoint('me/onenote/notebooks/' + notebookid + '/sections')
                    sections = MSGRAPH.get(sectionsendpoint).json()
                    
                    # check if section exists
                    sectionid = ""
                    for section in sections["value"]:
                        if setname == section["displayName"]:
                            sectionid = section["id"]
                            break

                            
                    # create new section if needed
                    if "" == sectionid:
                        print("... create new section: " + setname)
                        data = {"displayName": setname}
                        res = MSGRAPH.post(endpoint=sectionsendpoint, data=json.dumps(data))
                        if 201 != res.status_code:
                            print(res.json())
                            return "failed with status code: " + str(res.status_code)
                        resjson = res.json()
                        sectionid = resjson["id"]
                        
                    pageendpoint = "me/onenote/sections/" + sectionid + "/pages"

                    # loop over all cards
                    cardnr = 1
                    for card in root.findall('card'):
                        title        = card.get('title')
                        last_learned = card.get('last_learned')
                        level        = card.get('level')
                        source       = card.get('source')
                        tags         = card.get('tags')
                        content      = card.get('content')
                        images       = card.find('images')
                        
                        # check consistency
                        if int(cardnr) != int(card.get('position')):
                            return "Some cards seem to be missing in " + setname + ". Expected: " + str(cardnr) + " Got: " + str(card.get('position'));
    
                        
                        # create an image dictionary
                        images_dict  = dict()
                        for image in images.findall('image'):
                            key = image.get("alt")
                            value = image.get("url")
                            images_dict[key] = value

                        # create page for every card
                        boundary = "pageboundary34545" # some boundary
                        header = {'Content-Type' : 'multipart/form-data; boundary='+boundary} # override default header
                        data = createpage(title, content, images_dict, cardsetid, cardnr,  boundary)
                        res = MSGRAPH.post(endpoint=pageendpoint, data=data, headers=header)
                        if 201 != res.status_code:
                            print(res.json())
                            return "failed with status code: " + str(res.status_code)
                        
                        # increase card number
                        cardnr += 1
 
 
    # everything is fine      
    return "success!"

## Bottle and Microsoft Graphrest

In [ ]:
# user interface
bottle.TEMPLATE_PATH = ['./microsoft_graph_authenticator/static']
@bottle.route('/')
@bottle.view('homepage.html')
def homepage():
    """Render the home page."""
    return {'sample': 'graphrest'}

@bottle.route('/login')
def login():
    """Prompt user to authenticate."""
    MSGRAPH.login('/result')

@bottle.route('/login/authorized')
def authorized():
    """Handler for the application's Redirect Uri."""
    MSGRAPH.redirect_uri_handler()

@bottle.route('/result')
@bottle.view('result.html')
def result():
    return {'result': importcards()}

@bottle.route('/static/<filepath:path>')
def server_static(filepath):
    """Handler for static files, used with the development server."""
    root_folder = os.path.abspath(os.path.dirname(__file__))
    return bottle.static_file(filepath, root=os.path.join(root_folder, 'static'))

if __name__ == '__main__':
    bottle.run(app=bottle.app(), server='wsgiref', host='localhost', port=5000)